In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [2]:
# path to playlist file
root_path = !pwd
root_path = str(root_path[0])

In [3]:
# filename 
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [4]:
# path + filename
fpath_name = f"{filepath}{filename}"
# print(fpath_name)

In [5]:
# Open file of playlist and obtain fields of json file 
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

In [6]:
# # Explore fields in json file
# print("File keys: ", list(data.keys()))
# print("info: ",data[list(data.keys())[0]])
# #print(f"{list(data.keys())[1]} : ",data[list(data.keys())[1]])
# print("playlists: ",list(data['playlists'][0].keys()))
# print("tracks: ", list(data['playlists'][0]['tracks'][0].keys()))

In [7]:
# Extract track features
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

In [8]:
# Construct keys of playlist
keys = list(keys)
keys.remove('tracks')
print(" Playlist keys: ", keys)

 Playlist keys:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [9]:
# Create dataframe with track and playlist info
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'], record_prefix = 'track_')
music_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


In [43]:
# Check column values 
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [44]:
subset_df = music_df

In [45]:
# Select indexes of playlists that have a description
# playlists_id_with_description = music_df[~music_df['playlist_description'].isna()]['playlist_pid'].unique()
# print(playlists_id_with_description)

In [46]:
# Working only with playlists that contain a "description"
# subset_df = music_df[music_df['playlist_pid'].isin(playlists_id_with_description)]
# print(subset_df['playlist_pid'].unique())

In [47]:
num_records = len(subset_df)
num_unique_records = len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique())
print('Number of table records: ', num_records)
print('Number of unique artist, song pairs: ', num_unique_records )
print('Number of repeated songs: ', num_records - num_unique_records )
print('Number of playlists: ', len(subset_df['playlist_pid'].unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

Number of table records:  67503
Number of unique artist, song pairs:  34250
Number of repeated songs:  33253
Number of playlists:  1000
Number of unique artists:  9722


In [48]:
# Function to select track_uri ONLY 100 records for now
def get_features(df):
    indexes = []
    audio_features = []
    track_popularity = []
    artist_genre = []
    artist_popularity = []
    album_popularity = []
    start = 0
    for row_index, row in df.iloc[0:100].iterrows():
        #print(row_index, row['track_track_uri'])
        track_uri = row['track_track_uri']
        artist_uri = row['track_artist_uri']
        album_uri = row['track_album_uri']
        if start == 0:
            keys = spotify.audio_features(tracks=track_uri)[0].keys()
        start += 1
    
        track_popularity.append(spotify.track(track_uri)['popularity'])
        
        indexes.append(row_index) 
        audio_features.append(spotify.audio_features(tracks=track_uri)[0].values())
        
        artist_results = spotify.artist(artist_uri)
        
        artist_genre.append(artist_results['genres'])
        artist_popularity.append(artist_results['popularity'])
        #print(artist_results['genres'], artist_results['popularity'])
        album_results = spotify.album(album_uri)
        album_popularity.append(album_results['popularity'])
        
        
    features_df = pd.DataFrame(audio_features, columns=keys)
    features_df['song_popularity'] = track_popularity
    features_df['artist_genre'] = artist_genre
    features_df['artist_popularity']  = artist_popularity
    features_df['album_popularity'] = album_popularity
    features_df['index'] = indexes
    features_df.set_index('index', inplace=True)
    return features_df

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [49]:
audio_sp_df = get_features(subset_df)
audio_sp_df.index.name = None
audio_sp_df.drop(['type', 'id', 'track_href', 'analysis_url'], inplace=True, axis=1)
audio_sp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,4,84,"[dance pop, pop]",80,78
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,4,21,"[pop, r&b]",87,17
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267,4,79,"[dance pop, pop]",79,77
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.539,0.846,8,-3.192,1,0.0354,0.00630,0.000002,0.1300,0.460,106.992,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,274097,4,1,"[k-pop, k-pop boy group, pop]",90,1
96,0.729,0.898,5,-1.029,1,0.0391,0.31500,0.000000,0.1070,0.517,126.977,spotify:track:24psBRmEw3kHjBGZfl1dmb,218475,4,0,"[k-pop, k-pop girl group]",41,0
97,0.609,0.246,8,-8.449,1,0.0376,0.78400,0.000000,0.0825,0.371,123.773,spotify:track:06L1apH8kLF47dbhZ4Zg9A,288993,4,0,[k-pop],66,0
98,0.763,0.658,5,-5.643,0,0.1470,0.02580,0.000000,0.0915,0.389,80.038,spotify:track:2qWgqPdW1OiAP8KSBH1b93,217861,4,0,[k-pop],66,0


In [50]:
# Parameters to use for the model
audio_sp_df.keys()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity'],
      dtype='object')

In [51]:
check_df = subset_df.iloc[0:100].merge(audio_sp_df, how='left' , left_on = 'track_track_uri', right_on='uri')
check_df.drop(['track_pos', 'uri', 'mode', 'playlist_duration_ms','playlist_num_albums','playlist_num_artists',  'track_artist_uri', 'track_album_uri', 'track_duration_ms','playlist_num_followers', 'playlist_num_edits', 'playlist_collaborative', 'playlist_modified_at', 'playlist_num_tracks'], inplace = True, axis=1)
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.000002,0.1300,0.460,106.992,274097,4,1,"[k-pop, k-pop boy group, pop]",90,1
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.000000,0.1070,0.517,126.977,218475,4,0,"[k-pop, k-pop girl group]",41,0
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.000000,0.0825,0.371,123.773,288993,4,0,[k-pop],66,0
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.000000,0.0915,0.389,80.038,217861,4,0,[k-pop],66,0


In [52]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,102.000000,102.000000,102.000000,102.000000,102.00000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000
mean,0.595734,0.743118,4.862745,-6.050078,0.09711,0.128729,0.085879,0.176809,0.564255,122.015667,251604.264706,3.911765,40.872549,66.764706,41.039216
std,0.163594,0.177005,3.394927,2.581350,0.09932,0.178909,0.249239,0.134189,0.215020,30.649682,93382.498980,0.447279,32.919245,14.645093,33.221406
min,0.084900,0.205000,0.000000,-14.522000,0.02550,0.000010,0.000000,0.026800,0.133000,60.018000,65306.000000,1.000000,0.000000,28.000000,0.000000
25%,0.480500,0.651000,2.000000,-7.734500,0.03920,0.003677,0.000000,0.090400,0.386750,99.068500,206520.000000,4.000000,0.000000,60.000000,0.000000
50%,0.609000,0.786500,5.000000,-5.575000,0.05495,0.036850,0.000002,0.134500,0.559000,120.000500,229613.500000,4.000000,51.500000,66.000000,54.500000
75%,0.716000,0.892250,8.000000,-4.279750,0.11475,0.206250,0.001167,0.223750,0.730250,143.989500,273706.000000,4.000000,70.750000,78.000000,70.000000
max,0.938000,0.976000,11.000000,-1.029000,0.50100,0.893000,0.991000,0.722000,0.965000,210.857000,658987.000000,4.000000,85.000000,90.000000,86.000000


In [53]:
# Make album popularity equal to artist popularity if album popularity is 0
check_df['album_popularity'] = np.where(check_df['album_popularity'] == 0, check_df['artist_popularity'], check_df['album_popularity'])
check_df['song_popularity'] = np.where(check_df['song_popularity'] == 0, check_df['artist_popularity'], check_df['song_popularity'])
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.000002,0.1300,0.460,106.992,274097,4,1,"[k-pop, k-pop boy group, pop]",90,1
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.000000,0.1070,0.517,126.977,218475,4,41,"[k-pop, k-pop girl group]",41,41
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.000000,0.0825,0.371,123.773,288993,4,66,[k-pop],66,66
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.000000,0.0915,0.389,80.038,217861,4,66,[k-pop],66,66


In [54]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,102.000000,102.000000,102.000000,102.000000,102.00000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000
mean,0.595734,0.743118,4.862745,-6.050078,0.09711,0.128729,0.085879,0.176809,0.564255,122.015667,251604.264706,3.911765,60.490196,66.764706,62.235294
std,0.163594,0.177005,3.394927,2.581350,0.09932,0.178909,0.249239,0.134189,0.215020,30.649682,93382.498980,0.447279,21.643258,14.645093,20.358751
min,0.084900,0.205000,0.000000,-14.522000,0.02550,0.000010,0.000000,0.026800,0.133000,60.018000,65306.000000,1.000000,1.000000,28.000000,1.000000
25%,0.480500,0.651000,2.000000,-7.734500,0.03920,0.003677,0.000000,0.090400,0.386750,99.068500,206520.000000,4.000000,51.000000,60.000000,55.250000
50%,0.609000,0.786500,5.000000,-5.575000,0.05495,0.036850,0.000002,0.134500,0.559000,120.000500,229613.500000,4.000000,66.500000,66.000000,69.000000
75%,0.716000,0.892250,8.000000,-4.279750,0.11475,0.206250,0.001167,0.223750,0.730250,143.989500,273706.000000,4.000000,78.000000,78.000000,77.000000
max,0.938000,0.976000,11.000000,-1.029000,0.50100,0.893000,0.991000,0.722000,0.965000,210.857000,658987.000000,4.000000,89.000000,90.000000,89.000000


In [55]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 101
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_artist_name     102 non-null    object 
 1   track_track_uri       102 non-null    object 
 2   track_track_name      102 non-null    object 
 3   track_album_name      102 non-null    object 
 4   playlist_name         102 non-null    object 
 5   playlist_pid          102 non-null    object 
 6   playlist_description  0 non-null      object 
 7   danceability          102 non-null    float64
 8   energy                102 non-null    float64
 9   key                   102 non-null    int64  
 10  loudness              102 non-null    float64
 11  speechiness           102 non-null    float64
 12  acousticness          102 non-null    float64
 13  instrumentalness      102 non-null    float64
 14  liveness              102 non-null    float64
 15  valence               1

In [56]:
# NLP workflow
# Need to vectorize string fields
# Use all string fields and combine them in a column containing all the words 

In [57]:
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', ' ').replace('_', ' ') for i in x]))
# check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ''.join([i.replace('', '_') for i in x]))

In [58]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.000002,0.1300,0.460,106.992,274097,4,1,k pop k pop boy group pop,90,1
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.000000,0.1070,0.517,126.977,218475,4,41,k pop k pop girl group,41,41
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.000000,0.0825,0.371,123.773,288993,4,66,k pop,66,66
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.000000,0.0915,0.389,80.038,217861,4,66,k pop,66,66


In [59]:
label_columns = ['track_track_name', 'artist_genre']
check_df[label_columns]

,track_track_name,artist_genre
0,Lose Control (feat. Ciara & Fat Man Scoop),dance pop hip hop hip pop neo soul pop rap r&b...
1,Toxic,dance pop pop
2,Crazy In Love,pop r&b
3,Rock Your Body,dance pop pop
4,It Wasn't Me,dance pop pop rap reggae fusion
...,...,...
97,Spring Day,k pop k pop boy group pop
98,Ah-Choo,k pop k pop girl group
99,BREATHE,k pop
100,FXXK WIT US,k pop


In [85]:
# Combine 5 fields into one
# string_field = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
string_field = check_df.artist_genre
string_field = string_field.replace({"r\&b": "rhythm blues"}, regex = True)
string_field = string_field.replace({"[^A-Za-z ]+": ""}, regex = True)
#print('last song: ',string_field.tail())
# test_song = string_field[0:int(len(string_field)*0.4)]
# print(test_song)
string_field = string_field[int(len(string_field)*0.4):]
string_field

40     alternative metal modern rock pop punk punk ro...
41                              dance pop girl group pop
42                                      canadian pop pop
43     alternative metal modern rock neo mellow neon ...
44                                      canadian pop pop
                             ...                        
97                             k pop k pop boy group pop
98                                k pop k pop girl group
99                                                 k pop
100                                                k pop
101                                                k pop
Name: artist_genre, Length: 62, dtype: object

In [86]:
for i in string_field:
    if 'k p' in i:
        print(i)

alternative metal modern rock pop punk punk rock socal pop punk
alternative metal modern rock neo mellow neon pop punk pop punk pop rock post grunge
lilith neo mellow piano rock pop rock post teen pop
neon pop punk piano rock pop punk
neon pop punk pop punk
neon pop punk pop punk
alternative rock grunge pop rock post grunge rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
album rock canadian metal classic canadian rock classic rock hard rock progressive rock rock
k pop k rap korean

In [87]:
#check_df['All_text'] = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
check_df['All_text'] = check_df.artist_genre
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.0471,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62,dance pop hip hop hip pop neo soul pop rap r&b...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.2420,0.924,143.040,198800,4,84,dance pop pop,80,78,dance pop pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17,pop r&b
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.0521,0.817,100.972,267267,4,79,dance pop pop,79,77,dance pop pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.3130,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1,dance pop pop rap reggae fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.1300,0.460,106.992,274097,4,1,k pop k pop boy group pop,90,1,k pop k pop boy group pop
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.1070,0.517,126.977,218475,4,41,k pop k pop girl group,41,41,k pop k pop girl group
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.0825,0.371,123.773,288993,4,66,k pop,66,66,k pop
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.0915,0.389,80.038,217861,4,66,k pop,66,66,k pop


In [88]:
# Imports
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
# import nltk
# from nltk.stem import WordNetLemmatizer

In [90]:
# stopwords = nltk.corpus.stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# nltk.download('stopwords')

In [91]:
vectorizer = TfidfVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(string_field)
print(X_train)
print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")

  (0, 44)	0.3273854100875315
  (0, 39)	0.7396816406496044
  (0, 35)	0.28914184967058393
  (0, 42)	0.26129679661398425
  (0, 29)	0.29755535790715615
  (0, 28)	0.1699408259544881
  (0, 1)	0.27639059906357677
  (1, 18)	0.47605944337483724
  (1, 14)	0.5125139514093866
  (1, 10)	0.5125139514093866
  (1, 35)	0.49802239467228177
  (2, 6)	0.5726397480556149
  (2, 35)	0.8198071230154089
  (3, 19)	0.2904365465760451
  (3, 36)	0.24066175202599463
  (3, 32)	0.2537542355359663
  (3, 27)	0.22000334576882344
  (3, 31)	0.2904365465760451
  (3, 39)	0.48132350405198926
  (3, 35)	0.42333648877768065
  (3, 42)	0.2550454469169002
  (3, 29)	0.2904365465760451
  (3, 28)	0.16587510626477356
  (3, 1)	0.2697781403188739
  (4, 6)	0.5726397480556149
  :	:
  (52, 0)	0.15457679697554003
  (52, 6)	0.40441040392137206
  (52, 42)	0.7848142985718041
  (52, 28)	0.17014112157577915
  (53, 2)	0.6015642271482949
  (53, 41)	0.6015642271482949
  (53, 25)	0.5255863023623706
  (54, 2)	0.5000367045597531
  (54, 41)	0.5000367045

In [92]:
print(f"{X_train.nnz / np.prod(X_train.shape):.3f}")
print(f"About {X_train.nnz / np.prod(X_train.shape)*100:.3f}\% of the entries in the matrix are non zero")

0.077
About 7.683\% of the entries in the matrix are non zero


In [93]:
true_k = 5

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(X_train)
print(model.labels_)
print(len(model.labels_))

[4 2 2 4 2 2 0 2 2 2 2 4 4 4 1 3 3 3 3 3 3 3 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 2 2 2 2 2 2 2]
62


In [105]:
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

Top terms per cluster
Cluster 0: tunes broadway british soundtrack rap blues rhythm jazz vocal korean 
Cluster 1: rock classic hard album canadian metal glam progressive soft gold 
Cluster 2: pop korean canadian group teen post girl dance candy boy 
Cluster 3: scorecore yacht german jazz invasion indie heartland hard grunge group 
Cluster 4: punk pop neon rock modern alternative piano metal socal grunge 


In [106]:
#print(list(test_song[:1]))
#print(list(test_song)[0])
X_test = vectorizer.transform(list(test_song))
#print(X_test)
print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")
# feature_names = vectorizer.get_feature_names_out()
# print(feature_names)
prediction = model.predict(X_test)
print(prediction)

n_samples: 40, n_features: 55
[2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 2 4 4 0 2 2 0 2 2 2 4 4 2 0 2 2
 2 0 1]


In [107]:
print(model.labels_)
print(prediction)

[4 2 2 4 2 2 0 2 2 2 2 4 4 4 1 3 3 3 3 3 3 3 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 2 4 4 0 2 2 0 2 2 2 4 4 2 0 2 2
 2 0 1]


In [108]:
# # Save machine learning model
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [109]:
# # load model only once
# with open('finalized_model.sav', 'rb') as fid:
#     model = pickle.load(fid)

In [110]:
check_df['ClusterPrediction'] = ""
check_df['All_text']

0      dance pop hip hop hip pop neo soul pop rap r&b...
1                                          dance pop pop
2                                                pop r&b
3                                          dance pop pop
4                        dance pop pop rap reggae fusion
                             ...                        
97                             k pop k pop boy group pop
98                                k pop k pop girl group
99                                                 k pop
100                                                k pop
101                                                k pop
Name: All_text, Length: 102, dtype: object

In [111]:
def cluster_predict(str_input):
    #print(test_song)
    if isinstance(str_input, str):
        # It is a single string element thus we cannot use list() and have to use brackets [] to transform to list
        X_test = vectorizer.transform([str_input])
        prediction = model.predict(X_test)
    else:
        # if is not a single string element so we can transform into a list directly using list()
        X_test = vectorizer.transform(list(str_input))
        prediction = model.predict(X_test)
    return int(prediction)

In [112]:
check_df['All_text'][0]

'dance pop hip hop hip pop neo soul pop rap r&b rap urban contemporary virginia hip hop'

In [113]:
cluster_predict(check_df['All_text'][0])

2

In [114]:
cluster_predict([check_df['All_text'][0]])

2

In [115]:
result = check_df['All_text'].apply(lambda x: cluster_predict(x))
print(result)

0      2
1      2
2      2
3      2
4      2
      ..
97     2
98     2
99     2
100    2
101    2
Name: All_text, Length: 102, dtype: int64


In [116]:
# for i in check_df['All_text']:
#     pred = cluster_predict(i)
#     print(i, pred)
check_df['ClusterPrediction'] = check_df['All_text'].apply(lambda x: cluster_predict(x))

In [117]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text,ClusterPrediction
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62,dance pop hip hop hip pop neo soul pop rap r&b...,2
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.924,143.040,198800,4,84,dance pop pop,80,78,dance pop pop,2
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.701,99.259,235933,4,21,pop r&b,87,17,pop r&b,2
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.817,100.972,267267,4,79,dance pop pop,79,77,dance pop pop,2
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1,dance pop pop rap reggae fusion,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,BTS,spotify:track:0WNGsQ1oAuHzNTk8jivBKW,Spring Day,You Never Walk Alone,korean,2,NaN,0.539,0.846,8,...,0.460,106.992,274097,4,1,k pop k pop boy group pop,90,1,k pop k pop boy group pop,2
98,Lovelyz,spotify:track:24psBRmEw3kHjBGZfl1dmb,Ah-Choo,Lovelyz8,korean,2,NaN,0.729,0.898,5,...,0.517,126.977,218475,4,41,k pop k pop girl group,41,41,k pop k pop girl group,2
99,LEE HI,spotify:track:06L1apH8kLF47dbhZ4Zg9A,BREATHE,SEOULITE,korean,2,NaN,0.609,0.246,8,...,0.371,123.773,288993,4,66,k pop,66,66,k pop,2
100,LEE HI,spotify:track:2qWgqPdW1OiAP8KSBH1b93,FXXK WIT US,SEOULITE,korean,2,NaN,0.763,0.658,5,...,0.389,80.038,217861,4,66,k pop,66,66,k pop,2


In [663]:
# for seed in range(5):
#     model = KMeans(
#         n_clusters=true_k,
#         max_iter=500,
#         n_init=1,
#         random_state=seed,
#     ).fit(X)
#     cluster_ids, cluster_sizes = np.unique(model.labels_, return_counts=True)
#     print(f"Number of elements asigned to each cluster: {cluster_sizes}")
# print()

In [118]:
def recommend_util(artist_name, song_name):
    
    # Predict category of input string category
    chosen_song_df = check_df.loc[(check_df['track_artist_name'] == artist_name) & (check_df['track_track_name'] == song_name)]
    str_input = chosen_song_df.track_track_name.str.cat(" " + chosen_song_df.artist_genre)
        
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    
    temp_df = check_df.loc[check_df['ClusterPrediction'] == prediction_inp]
    new_temp_df = temp_df.sample(5)
    
    return chosen_song_df[['track_artist_name', 'track_track_name']], new_temp_df[['track_artist_name', 'track_track_name']]

In [119]:
song_choice = 2
print(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
temp_df = check_df.loc[(check_df['track_artist_name'] == check_df['track_artist_name'][0]) & (check_df['track_track_name'] == check_df['track_track_name'][0])]
string_input = temp_df.track_track_name.str.cat(" " + temp_df.artist_genre)
string_input
prediction_inp = cluster_predict(string_input)
print(prediction_inp)

Beyoncé Crazy In Love
2


In [120]:
song_choice = 1
original_song , recommended_songs = recommend_util(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
print(original_song)
print('\n')
print(recommended_songs)

  track_artist_name track_track_name
1    Britney Spears            Toxic


    track_artist_name  track_track_name
101             Ailee   I Will Show You
33               Iyaz            Replay
98            Lovelyz           Ah-Choo
14             Cassie            Me & U
19    Destiny's Child  Jumpin', Jumpin'


In [228]:
# Example of how to pull metadata from a single track 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

Row = 5 

track_uri = subset_df.iloc[Row]['track_track_uri']
artist_uri = subset_df.iloc[Row]['track_artist_uri']
album_uri = subset_df.iloc[Row]['track_album_uri']

print('ARTIST: ', subset_df.iloc[Row]['track_artist_name'])
print('ALBUM: ', subset_df.iloc[Row]['track_album_name'])
print('SONG: ', subset_df.iloc[Row]['track_track_name'])
track_popularity = spotify.track(track_uri)['popularity']
print('track_popularity: ', track_popularity)
track_results = spotify.audio_features(tracks=track_uri)
print('audio_features: ', track_results[0])
artist_results = spotify.artist(artist_uri)
print('artist_genre: ', artist_results['genres'])
print('artist_popularity: ', artist_results['popularity'])
artist_albums_results = spotify.artist_albums(artist_uri)
#print(artist_albums_results)
album_results = spotify.album(album_uri)
print('album_popularity: ',album_results['popularity'])
print('album_keys: ', album_results.keys())
# print(artist_results['genres'])
# print(artist_results['popularity'])

ARTIST:  Ron Pope
ALBUM:  The Bedroom Demos
SONG:  A Drop In The Ocean
track_popularity:  67
audio_features:  {'danceability': 0.447, 'energy': 0.393, 'key': 5, 'loudness': -8.65, 'mode': 1, 'speechiness': 0.038, 'acousticness': 0.785, 'instrumentalness': 0, 'liveness': 0.28, 'valence': 0.564, 'tempo': 73.139, 'type': 'audio_features', 'id': '5JDcQAztvZTIkrWoZihgvC', 'uri': 'spotify:track:5JDcQAztvZTIkrWoZihgvC', 'track_href': 'https://api.spotify.com/v1/tracks/5JDcQAztvZTIkrWoZihgvC', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5JDcQAztvZTIkrWoZihgvC', 'duration_ms': 220239, 'time_signature': 3}
artist_genre:  ['neo mellow', 'piano rock', 'viral pop']
artist_popularity:  53
album_popularity:  59
album_keys:  dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])


In [229]:
# Work only with important features
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_duration_ms', 'playlist_description']

In [ ]:
artist_count_df = subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort=False).value_counts(sort=False).reset_index(name="artist_count")

In [ ]:
artist_count_df

In [ ]:
subset_df[['playlist_name', 'track_artist_name']]

In [ ]:
subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort = False).value_counts(sort=False).reset_index(name="artist_count").info()

In [ ]:
music_df = pd.merge(subset_df, artist_count_df,  how='left', on = ['playlist_name','track_artist_name'])

In [ ]:
music_df

In [ ]:
# subset_df.join(.sum(), on='playlist_name', rsuffix='_count')
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1)

In [ ]:
important_features.append("artist_count")

In [ ]:
music_df[important_features]

In [ ]:
music_df['artist_fraction'] =  music_df['artist_count'] / music_df['playlist_num_tracks'] 
music_df

In [ ]:
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).value_counts() 

In [ ]:
vc = music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x), axis=1).value_counts()
vc[vc > 1].sum() - len(vc[vc > 1])

In [ ]:
# for i in df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique():
# #     print(i.split(',.'))
#     if len(i.split(',.')) == 2:
#         print(i)

In [ ]:
np.unique(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_track_name'])

In [ ]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [ ]:
music_df[music_df['playlist_num_tracks'] == 20][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][important_features].groupby('playlist_name')['track_artist_name'].value_counts()